In [37]:
import numpy as np
import pandas as pd
import MDAnalysis as mda
import os
import csv
import matplotlib.pyplot as plt
from multiprocessing import Pool, TimeoutError
from concurrent.futures import ThreadPoolExecutor, as_completed
import seaborn as sns
import natsort
import regex as re
from scipy import stats

In [2]:
import time

def init_pool():
    np.random.seed(1)


def cal_edges(arg):
    '''
    This function is designed for multiprocessing Pool.map function.
    
    input arg: argument from Pool.map() function.
    return: a numpy array contains all distances between defined atoms of secondary lipids
    '''
    def cal_dist_(arr, length):
        '''
        This function is used to calculate 2d distance between all input points with pbc conditions.
        
        input arr: a numpy array contains 2d coordinates of certain amount of points.
        input length: length of one dimension of pbc boundary needs to be jumped.
        return the distance of all points.
        '''
        rst = np.array([])
        for i in range(arr.shape[0] - 1):
            dist_ = arr[i] - arr[i+1: ]
            # jump pbc
            dist_[dist_ > length/2] -= length/2
            dist_[dist_ < -length/2] += length/2
            rst = np.hstack((rst, np.sqrt(np.sum(dist_ ** 2, axis=1))))
        #time.sleep(1)
        return np.array(rst).flatten()
    sec_lipid_headgroup_select_str = 'name P and resname {}'.format(sec_lipid_name)
    Lx = arg.dimensions[0]
    #for frame in u.trajectory[arg.frame:arg.frame+1]:
    p_dict = {'upper':u.select_atoms(sec_lipid_headgroup_select_str + ' and resid 1-100').positions[:, :2],
              'lower':u.select_atoms(sec_lipid_headgroup_select_str + ' and resid 101-200').positions[:, :2]
             }

    if 'name P' in sec_lipid_headgroup_select_str:
        p_dict['upper'] = p_dict['upper'].reshape(sec_lipid['number'], 2)
        p_dict['lower'] = p_dict['lower'].reshape(sec_lipid['number'], 2)
    else:
        # Get COM of headgroup atoms
        p_dict['upper'] = np.average(p_dict['upper'].reshape(sec_lipid['number'], P_index, 2), axis = 0)
        p_dict['lower'] = np.average(p_dict['lower'].reshape(sec_lipid['number'], P_index, 2), axis = 0)
        
    rst = np.array([cal_dist_(p_dict['upper'], Lx), cal_dist_(p_dict['lower'], Lx)]).flatten()
    time.sleep(.1)
    return rst

def get_files(pre = 'gamma', ext = 'dcd'):
    ''' This function returns files under current folder with defined prefix and extension.
    input pre: string, prefix of wanted files
    input ext: string, extension of wanted files.
    return: a list of file names.
    '''
    result = []
    for file_ in os.scandir():
        if file_.is_file() and re.search(r'^(%s)'%pre, file_.name) and re.search(r'(%s)$'%ext, file_.name):
            result.append(file_.name)
    result = natsort.natsorted(result, alg=natsort.FLOAT)        
    return result

def get_dirs(pre = '65', ext = 'input'):
    ''' This function returns files under current folder with defined prefix and extension.
    input pre: string, prefix of wanted files
    input ext: string, extension of wanted files.
    return: a list of file names.
    '''
    result = []
    for file_ in os.scandir():
        if file_.is_dir() and re.search(r'^(%s)'%pre, file_.name) and re.search(r'(%s)$'%ext, file_.name):
            result.append(file_.name)
    result = natsort.natsorted(result, alg=natsort.FLOAT)        
    return result

In [3]:
def repair_bilayer(u, frame):
    """ This function repairs bilayer based on the status of water. If there's only one layer of water,
    bilayer must be broken.
    input:
        1. u: universe of specific frame.
        2. frame: frame of current frame stores dimension.
    no returns, modify coordinates of current frame to make bilayer whole.
    """
    def aveNearTwo(array: np.array) -> np.array:
        a = array[:-1]
        b = array[1:]
        return ( a + b ) /2
    water_coors = u.select_atoms('resname TIP3').positions
    dens_, coors_ = np.histogram(water_coors[:,2], bins=30)

    # if there's only one layer of water bulk, create a blank space
    if 0 not in dens_:
        water_coors[:,2][water_coors[:,2] < 0 ] += frame.dimensions[2]  # move water lower than 0 plane one z pbc up.
        dens_, coors_ = np.histogram(water_coors[:,2], bins=30)
        coors_ = aveNearTwo(coors_)
        blank_z_coor = np.average(coors_[ dens_ == 0 ])
        u.atoms.translate([0, 0, -blank_z_coor]).wrap()

    elif dens_[0] != 0 and dens_[-1] != 0:
        coors_ = aveNearTwo(coors_)
        blank_z_coor = np.average(coors_[ dens_ == 0 ])
        u.atoms.translate([0, 0, -blank_z_coor]).wrap()
        # If there are two layers of water bulk, find the center z coor of blank bulk.

    elif dens_[0] == 0 or dens_[-1] == 0:
        coors_ = aveNearTwo(coors_)
        blank_z_coor = np.average(coors_[ dens_ == 0 ])
        u.atoms.translate([0, 0, -blank_z_coor]).wrap()

    # move bilayer to center
    water_center = np.average(u.select_atoms('resname TIP3').positions[:,2])
    u.atoms.translate([0, 0, -water_center]).wrap()       

## Area Compressibility Modulus with Auto-correlation function

In [ ]:
def corr_time(array):
    """ 
    This function returns the autocorrelation function of its input.
    input array: a numeric array.
    return: index of 0 point in its input array, numeric.

    Algorithms from https://doi.org/10.33011/livecoms.1.1.5067.
    Returns:
    c_f: aotucorrelation function. Can be used for plotting.
    n_ind: number of independent samples. 
    variables:
    _mean, mean of its input 
    _variance, variance of its input
    c_f, average amount of autocorrelation function between snapshots separated by t'.
    _length, length of its input array which is the length of simulation.
    """
    def first_nearest_zero(array):
        """ returns the index and value of the first point whose y value changes sign in the array (+ -> - ).
        """

        for i, value in enumerate(array):
            if array[i] > 0 and array[i+1] < 0:
                return [i, value]

    _mean = np.average(array)
    _variance = np.var(array)
    c_f = []
    _length = len(array)

    # looping through its input array for different t'.
    for t_prime in range(1, _length):
        c_f += [ np.average((array[:-t_prime]-_mean)*(array[t_prime:]-_mean)) / _variance]

    x_zero, y_zero = first_nearest_zero(c_f)
    n_ind = _length//(1+2*np.sum(c_f[:x_zero]))

    return c_f, n_ind

def chunk(array, arr_size=1):
    """ returns a list contains arr_size of sublists of its input array
    """
    result = []
    arr_length = len(array)
    sub_array = range(0, arr_length, int( (arr_length - 1) / arr_size))
    for i in range(len(sub_array) - 1):
        result += [array[ sub_array[i] : sub_array[i+1]]]

    return result

In [ ]:
os.chdir('/work/hung_group/xu.jiam/liposomes/namd_sims/DOPC+PA/100DSPA/namd')

In [ ]:
dcd_files = get_files()

In [ ]:
surf_tensions = [-7, 0, 7, 15]
all_apl_list = []
all_n_samples = []
for st in surf_tensions:
    dcd_gamma = [ x for x in dcd_files if x.startswith('gamma'+str(st)) ]
    if 'para' in ';'.join(dcd_gamma):
        dcd_gamma = [ x for x in dcd_gamma if 'para' in x ]
    
    u = mda.Universe('step5_input.psf', dcd_gamma)
    boxx = []
    
    for frame in u.trajectory:
        boxx.append(u.dimensions[0])
    
    _,n_samples = corr_time(boxx)
    all_n_samples.append(n_samples)
    
    all_apl_list.append(np.array(boxx) ** 2 / 100)

In [ ]:
kas=[]
min_n_samples = min(all_n_samples)
all_apl_list_spt = [ chunk(x, min_n_samples) for x in all_apl_list ]
## Calculate mean apl for each chunk.
all_apl_list_spt = [ np.average(x) for y in all_apl_list_spt for x in y ]
all_apl_list_spt = np.array(all_apl_list_spt).reshape((len(surf_tensions), -1))
# Loop each column of different gamma apls and apply linear regression
area_strain = np.vstack((np.zeros(5),all_apl_list_spt[1:,:]/all_apl_list_spt[0,:]-1))
for i in range(int(min_n_samples)):
    ka = stats.linregress(area_strain[:,i], surf_tensions)
    kas.append(ka.slope)
#return [np.average(kas), np.std(kas)/np.sqrt(min_n_samples)]

In [ ]:
np.average(kas), np.std(kas)

## Calculate Head group distance distributions

It can process multiple folders using multiple cores.

In [48]:
sec_lipid_name = 'DSPE'
surf_tensions = [-7, 0, 7, 15]
PA_names = ['DEPA','DGPA','DLPA','DMPA','DNPA','DOPA','DPPA','DSPA','DYPA','PLPA','POPA','PSPA','SLPA','SOPA','YOPA']
#PA_names = ['DLPA','DMPA','DPPA','DSPA']
DS_names = ['DSPC', 'DSPE', 'DSPG', 'DSPS', 'DSPA']
csv_data = []

In [ ]:
results = {}
for st in surf_tensions:
    results[st] = {}

for sec_lipid_name in DS_names[:]:
    os.chdir("/scratch/xu.jiam/liposomes/namd_sims/DOPC_DS/{}85/namd".format(sec_lipid_name))
    dcdfiles = get_files()
    #u = mda.Universe('step5_input.psf', 'gamma0.part001.dcd')
    
    for st in surf_tensions:
        
        results[st][sec_lipid_name] = []
        dcdfile = [ i for i in dcdfiles if i.startswith('gamma'+str(st)) ]
        dcdfile = [ i for i in dcdfile if 'para' not in i ]
        u = mda.Universe('step5_input.psf', dcdfile)

        sec_lipid = {'name':sec_lipid_name}
        sec_lipid['number'] = np.unique(u.select_atoms('resname {}'.format(sec_lipid_name)).resids).shape[0] // 2

        sec_resid = u.select_atoms('resname {}'.format(sec_lipid_name)).resids[0]
        P_index = np.where(u.select_atoms('resid {}'.format(sec_resid)).atoms.names == 'P')[0][0]
        sec_lipid_headgroup_select_str = '(name ' + ' or name '.join(u.select_atoms('resid {}'.format(sec_resid)).names[:P_index]) + ')' + ' and resname ' + sec_lipid['name']

        edges = np.array([])
        
        with ThreadPoolExecutor(20) as executor:
            timeout = 5 # set the timeout in seconds
            futures = [executor.submit(cal_edges, args) for args in u.trajectory[-500::5]]
            for future in as_completed(futures):
                try:
                    result = future.result(timeout=timeout)
                    results[st][sec_lipid_name].append(list(result))
                    #print(result)
                except TimeoutError:
                    print("TimeoutError Occured, moving to the next task")
                    continue
                except StopIteration:
                    print("StopIteration Occured, moving to the next task")

        #with Pool(20, initializer=init_pool) as pool:
        #    for result in pool.map(cal_edges, u.trajectory[-500:]):
        #for frame in u.trajectory[-500:]:
        #    result = cal_edges(frame)
        #        results[st].append(list(result))
        #        #edges = np.hstack((edges, results))
        
        print('surface '+str(st)+' is finished.', end='\r')
        
        

    print("Lipid {} is finished.".format(sec_lipid_name))


In [ ]:
bins_ = np.linspace(0, 60, 20)
csv_header = ['composition', 'gamma'] + list((rst_bins_[1:] + rst_bins_[:-1])/2)

for st in surf_tensions[:]:
    for lipid_ in PA_names[:]:
        freq_, rst_bins_ = np.histogram(results[st][lipid_], bins=bins_)
        prob_ = freq_/sum(freq_)
        csv_data.append( ['85'+'-DOPC-'+lipid_, st] + list(prob_))

In [49]:
os.getcwd()

'/work/hung_group/xu.jiam/liposomes/namd_sims/DOPC+PA'

In [ ]:
os.chdir("/scratch/xu.jiam/liposomes/namd_sims/Results")

In [ ]:
with open('HD_hist_PA.csv', 'w', encoding='UTF8', newline='') as f:
    writer = csv.writer(f)

    # write the header
    writer.writerow(csv_header)

    # write multiple rows
    writer.writerows(csv_data)

---

In [ ]:
os.chdir('/scratch/xu.jiam/liposomes/namd_sims/Results')
fig, axs = plt.subplots(nrows=4, ncols=1, figsize=(4,16))
fig.tight_layout(pad=2.5, rect=[0, 0.03, 1, 0.95])
ii = 0
bins_ = np.linspace(0, 60, 20)
for r in range(1):
    for c in range(4):
        sns.histplot(ax = axs[c], data = np.array(results[surf_tensions[ii]]['DSPC']).flatten(), bins=bins_, color = 'red', stat='probability',   alpha=1, label = 'DSPC')
        sns.histplot(ax = axs[c], data = np.array(results[surf_tensions[ii]]['DSPE']).flatten(), bins=bins_, color = 'gray', stat='probability',   alpha=1)#, label = 'DPPA')
        sns.histplot(ax = axs[c], data = np.array(results[surf_tensions[ii]]['DSPG']).flatten(), bins=bins_, color = 'gray', stat='probability',   alpha=1)#, label = 'DSPA')
        sns.histplot(ax = axs[c], data = np.array(results[surf_tensions[ii]]['DSPS']).flatten(), bins=bins_, color = 'gray', stat='probability',   alpha=1)#, label = 'DLPA') #, bins=15, stat='probability')#, kde=True)
        sns.histplot(ax = axs[c], data = np.array(results[surf_tensions[ii]]['DSPA']).flatten(), bins=bins_, color = 'gray', stat='probability',   alpha=1)#, label = 'DSPS')
        #axs[c].set_title('gamma = '+ str(surf_tensions[ii]))
        #axs[r , c].set_ylim(0, 0.15)
        axs[c].set_xlim(0,60)
        ii += 1

plt.suptitle("Distances between headgroups of four secondary lipids for 65% systems.")
plt.legend(loc='lower center', bbox_to_anchor=(0.5,-.25), ncol=5)#, labels=['DSPC','DSPE'])#,'DSPG','DSPS'])
plt.savefig('DSPC.jpg', dpi=300)
plt.close()

os.chdir('/scratch/xu.jiam/liposomes/namd_sims/Results')
fig, axs = plt.subplots(nrows=4, ncols=1, figsize=(4,16))
fig.tight_layout(pad=2.5, rect=[0, 0.03, 1, 0.95])
ii = 0
bins_ = np.linspace(0, 60, 20)
for r in range(1):
    for c in range(4):
        sns.histplot(ax = axs[c], data = np.array(results[surf_tensions[ii]]['DSPE']).flatten(), bins=bins_, color = 'red', stat='probability',   alpha=1, label = 'DSPE')
        sns.histplot(ax = axs[c], data = np.array(results[surf_tensions[ii]]['DSPC']).flatten(), bins=bins_, color = 'gray', stat='probability',   alpha=1)#, label = 'DMPA')
        sns.histplot(ax = axs[c], data = np.array(results[surf_tensions[ii]]['DSPG']).flatten(), bins=bins_, color = 'gray', stat='probability',   alpha=1)#, label = 'DSPA')
        sns.histplot(ax = axs[c], data = np.array(results[surf_tensions[ii]]['DSPS']).flatten(), bins=bins_, color = 'gray', stat='probability',   alpha=1)#, label = 'DLPA') #, bins=15, stat='probability')#, kde=True)
        sns.histplot(ax = axs[c], data = np.array(results[surf_tensions[ii]]['DSPA']).flatten(), bins=bins_, color = 'gray', stat='probability',   alpha=1)#, label = 'DSPS')
        #axs[c].set_title('gamma = '+ str(surf_tensions[ii]))
        #axs[r , c].set_ylim(0, 0.15)
        axs[c].set_xlim(0,60)
        ii += 1

plt.suptitle("Distances between headgroups of four secondary lipids for 65% systems.")
plt.legend(loc='lower center', bbox_to_anchor=(0.5,-.25), ncol=5)#, labels=['DSPC','DSPE'])#,'DSPG','DSPS'])
plt.savefig('DSPE.jpg', dpi=300)
plt.close()

os.chdir('/scratch/xu.jiam/liposomes/namd_sims/Results')
fig, axs = plt.subplots(nrows=4, ncols=1, figsize=(4,16))
fig.tight_layout(pad=2.5, rect=[0, 0.03, 1, 0.95])
ii = 0
bins_ = np.linspace(0, 60, 20)
for r in range(1):
    for c in range(4):
        sns.histplot(ax = axs[c], data = np.array(results[surf_tensions[ii]]['DSPG']).flatten(), bins=bins_, color = 'red', stat='probability',   alpha=1, label = 'DSPG')
        sns.histplot(ax = axs[c], data = np.array(results[surf_tensions[ii]]['DSPE']).flatten(), bins=bins_, color = 'gray', stat='probability',   alpha=1)#, label = 'DMPA')
        sns.histplot(ax = axs[c], data = np.array(results[surf_tensions[ii]]['DSPC']).flatten(), bins=bins_, color = 'gray', stat='probability',   alpha=1)#, label = 'DPPA')
        sns.histplot(ax = axs[c], data = np.array(results[surf_tensions[ii]]['DSPS']).flatten(), bins=bins_, color = 'gray', stat='probability',   alpha=1)#, label = 'DLPA') #, bins=15, stat='probability')#, kde=True)
        sns.histplot(ax = axs[c], data = np.array(results[surf_tensions[ii]]['DSPA']).flatten(), bins=bins_, color = 'gray', stat='probability',   alpha=1)#, label = 'DSPS')
        #axs[c].set_title('gamma = '+ str(surf_tensions[ii]))
        #axs[r , c].set_ylim(0, 0.15)
        axs[c].set_xlim(0,60)
        ii += 1

plt.suptitle("Distances between headgroups of four secondary lipids for 65% systems.")
plt.legend(loc='lower center', bbox_to_anchor=(0.5,-.25), ncol=5)#, labels=['DSPC','DSPE'])#,'DSPG','DSPS'])
plt.savefig('DSPG.jpg', dpi=300)
plt.close()

os.chdir('/scratch/xu.jiam/liposomes/namd_sims/Results')
fig, axs = plt.subplots(nrows=4, ncols=1, figsize=(4,16))
fig.tight_layout(pad=2.5, rect=[0, 0.03, 1, 0.95])
ii = 0
bins_ = np.linspace(0, 60, 20)
for r in range(1):
    for c in range(4):
        sns.histplot(ax = axs[c], data = np.array(results[surf_tensions[ii]]['DSPS']).flatten(), bins=bins_, color = 'red', stat='probability',   alpha=1, label = 'DSPS') #, bins=15, stat='probability')#, kde=True)
        sns.histplot(ax = axs[c], data = np.array(results[surf_tensions[ii]]['DSPE']).flatten(), bins=bins_, color = 'gray', stat='probability',   alpha=1)#, label = 'DMPA')
        sns.histplot(ax = axs[c], data = np.array(results[surf_tensions[ii]]['DSPG']).flatten(), bins=bins_, color = 'gray', stat='probability',   alpha=1)#, label = 'DPPA')
        sns.histplot(ax = axs[c], data = np.array(results[surf_tensions[ii]]['DSPC']).flatten(), bins=bins_, color = 'gray', stat='probability',   alpha=1)#, label = 'DSPA')
        sns.histplot(ax = axs[c], data = np.array(results[surf_tensions[ii]]['DSPA']).flatten(), bins=bins_, color = 'gray', stat='probability',   alpha=1)#, label = 'DSPS')
        #axs[c].set_title('gamma = '+ str(surf_tensions[ii]))
        #axs[r , c].set_ylim(0, 0.15)
        axs[c].set_xlim(0,60)
        ii += 1

plt.suptitle("Distances between headgroups of four secondary lipids for 65% systems.")
plt.legend(loc='lower center', bbox_to_anchor=(0.5,-.25), ncol=5)#, labels=['DSPC','DSPE'])#,'DSPG','DSPS'])
plt.savefig('DSPS.jpg', dpi=300)
plt.close()


os.chdir('/scratch/xu.jiam/liposomes/namd_sims/Results')
fig, axs = plt.subplots(nrows=4, ncols=1, figsize=(4,16))
fig.tight_layout(pad=2.5, rect=[0, 0.03, 1, 0.95])
ii = 0
bins_ = np.linspace(0, 60, 20)
for r in range(1):
    for c in range(4):
        sns.histplot(ax = axs[c], data = np.array(results[surf_tensions[ii]]['DSPA']).flatten(), bins=bins_, color = 'red', stat='probability',   alpha=1, label = 'DSPA') #, bins=15, stat='probability')#, kde=True)
        sns.histplot(ax = axs[c], data = np.array(results[surf_tensions[ii]]['DSPE']).flatten(), bins=bins_, color = 'gray', stat='probability',   alpha=1)#, label = 'DMPA')
        sns.histplot(ax = axs[c], data = np.array(results[surf_tensions[ii]]['DSPG']).flatten(), bins=bins_, color = 'gray', stat='probability',   alpha=1)#, label = 'DPPA')
        sns.histplot(ax = axs[c], data = np.array(results[surf_tensions[ii]]['DSPC']).flatten(), bins=bins_, color = 'gray', stat='probability',   alpha=1)#, label = 'DSPA')
        sns.histplot(ax = axs[c], data = np.array(results[surf_tensions[ii]]['DSPS']).flatten(), bins=bins_, color = 'gray', stat='probability',   alpha=1)#, label = 'DSPS')
        #axs[c].set_title('gamma = '+ str(surf_tensions[ii]))
        #axs[r , c].set_ylim(0, 0.15)
        axs[c].set_xlim(0,60)
        ii += 1

plt.suptitle("Distances between headgroups of four secondary lipids for 65% systems.")
plt.legend(loc='lower center', bbox_to_anchor=(0.5,-.25), ncol=5)#, labels=['DSPC','DSPE'])#,'DSPG','DSPS'])
plt.savefig('DSPA.jpg', dpi=300)
plt.close()

In [ ]:
os.getcwd()

In [ ]:
fig, axs = plt.subplots(nrows=2, ncols=2)
fig.tight_layout(pad=2.5, rect=[0, 0.03, 1, 0.95])
ii = 0
bins_ = np.linspace(0, 60, 20)

for r in range(2):
    for c in range(2):
        sns.histplot(ax = axs[r, c], data = np.array(results[surf_tensions[ii]]['DSPE']).flatten(), bins=bins_, stat='probability', alpha=0.3)
        ii += 1    

In [ ]:
from mpl_toolkits.mplot3d import Axes3D

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

# Define the bin edges for x, y, and z
x_bins = np.linspace(0, 60, 20)
y_bins = np.linspace(0, 60, 20)
z_bins = np.linspace(0, 60, 20)

hist, x_edges, y_edges, z_edges = np.histogram2d(x, y, z, bins=(x_bins, y_bins, z_bins))

# Flatten the histogram for plotting
x_pos, y_pos, z_pos = np.meshgrid(x_edges[:-1], y_edges[:-1], z_edges[:-1], indexing="ij")
x_pos = x_pos.ravel()
y_pos = y_pos.ravel()
z_pos = z_pos.ravel()

# Plot the 3D bars
ax.bar3d(x_pos, y_pos, z_pos, dx=x_edges[1]-x_edges[0], dy=y_edges[1]-y_edges[0], dz=z_edges[1]-z_edges[0], color='b', zsort='average')


## Calculate Order Parameters

Note:
Parallel computing is not tuned properly, the results are different from signle thread ones.

In [ ]:
def Sc(v1, v2):
    norm = np.array([0, 0, 1])
    """ returns order parameter of two numpy arrays contain x-y-z coordinates.
    inputs"""
    cos_theta = (np.dot(v1 - v2, norm) / (np.linalg.norm(v1 - v2, axis=2)))
    return 0.5 * (3 * cos_theta **2 - 1)

def repair_bilayer(u, frame):
    """ This function repairs bilayer based on the status of water. If there's only one layer of water,
    bilayer must be broken.
    input:
        1. u: universe of specific frame.
        2. frame: frame of current frame stores dimension.
    no returns, modify coordinates of current frame to make bilayer whole.
    """
    print(frame.frame)
    def aveNearTwo(array: np.array) -> np.array:
        a = array[:-1]
        b = array[1:]
        return ( a + b ) /2
    #for frame_ in u.trajectory[frame.frame:frame.frame+1]:
    water_coors = u.select_atoms('resname TIP3').positions
    dens_, coors_ = np.histogram(water_coors[:,2], bins=30)

    # if there's only one layer of water bulk, create a blank space
    if 0 not in dens_:
        water_coors[:,2][water_coors[:,2] < 0 ] += frame.dimensions[2]  # move water lower than 0 plane one z pbc up.
        dens_, coors_ = np.histogram(water_coors[:,2], bins=30)
        coors_ = aveNearTwo(coors_)
        blank_z_coor = np.average(coors_[ dens_ == 0 ])
        u.atoms.translate([0, 0, -blank_z_coor]).wrap()

    elif dens_[0] != 0 and dens_[-1] != 0:
        coors_ = aveNearTwo(coors_)
        blank_z_coor = np.average(coors_[ dens_ == 0 ])
        u.atoms.translate([0, 0, -blank_z_coor]).wrap()
        # If there are two layers of water bulk, find the center z coor of blank bulk.

    elif dens_[0] == 0 or dens_[-1] == 0:
        coors_ = aveNearTwo(coors_)
        blank_z_coor = np.average(coors_[ dens_ == 0 ])
        u.atoms.translate([0, 0, -blank_z_coor]).wrap()

    # move bilayer to center
    water_center = np.average(u.select_atoms('resname TIP3').positions[:,2])
    u.atoms.translate([0, 0, -water_center]).wrap()

def Sc(v1, v2):
    norm = np.array([0, 0, 1])
    """ returns order parameter of two numpy arrays contain x-y-z coordinates.
    inputs"""
    cos_theta = (np.dot(v1 - v2, norm) / (np.linalg.norm(v1 - v2, axis=2)))
    return 0.5 * (3 * cos_theta **2 - 1)    
    
def cal_op(arg):
    
    results = op_results_empty.copy()
    
    try:
        for frame in u.trajectory[arg.frame:arg.frame+1]:
            #print(frame.frame)
            #repair_bilayer(u, frame)
        
            for lip_name in lipid_names:
                ## Get positions of carbons in tails.
                c1_pos = u.select_atoms(c_names_dict[lip_name]['c1_string']).positions
                c2_pos = u.select_atoms(c_names_dict[lip_name]['c2_string']).positions
                ## Handle these positions. there are n - 2 bonds among n carbon atoms. therefore, the size of
                ## order parameter would be n - 2.

                ## Loop through the first acyl
                y1 = u.select_atoms(c_names_dict[lip_name]['c1_string']).positions.reshape(( -1, len(c_names_dict[lip_name]['c1']), 3))
                y2 = u.select_atoms(c_names_dict[lip_name]['c2_string']).positions.reshape(( -1, len(c_names_dict[lip_name]['c2']), 3))

                results[lip_name][0].append(np.average(Sc(y1[:,:-2], y1[:,2:]), axis=0))
                results[lip_name][1].append(np.average(Sc(y2[:,:-2], y2[:,2:]), axis=0))
            plt.plot(c1_pos[:,0], c1_pos[:,2], 'x')
            #plt.show()
        #print("Frame processed")
        return results
    
    except:
        return (None, None)


#return c_names_dict, op_results
    
    

In [ ]:
folders = get_dirs()
acyl_cnames = [ 'C'+str(x+2) for x in range(22)]
csv_header = ['composition', 'gamma', 'lipid', 'SN'] +  acyl_cnames
csv_data = []

In [ ]:
y1 = u.select_atoms(c_names_dict[lip_name]['c1_string']).positions.reshape(( -1, len(c_names_dict[lip_name]['c1']), 3))
y2 = u.select_atoms(c_names_dict[lip_name]['c2_string']).positions.reshape(( -1, len(c_names_dict[lip_name]['c2']), 3))

#results[lip_name][0].append(np.average(Sc(y1[:,:-2], y1[:,2:]), axis=0))

In [ ]:
surf_tensions = [-7, 0]#, 7, 15]

In [ ]:
csv_data = []
os.chdir('/scratch/xu.jiam/liposomes/namd_sims/DOPC+PA/')
folders=get_dirs(pre='75', ext='input')
folders=['65-DOPC-DPPA-input']
TIMEOUT=3
for folder in folders:
    os.chdir(folder + '/namd/')
    dcdfiles = get_files()
    dcdfiles = [ i for i in dcdfiles if 'para' not in i ]
    ts = 2
    
    for st in surf_tensions:
        print('Processing gamma {}.'.format(st), end='\r')
        dcdfile = [ i for i in dcdfiles if i.startswith('gamma'+str(st)) ]
        u = mda.Universe("step5_input.psf", dcdfile)
        lipid_names = [ i for i in np.unique(u.residues.resnames) if i not in ['TIP3', 'SOD', 'CLA'] ]
        c_names_dict = {}
        op_results = {}
        op_results_overall = {}
        # Extract carbon atom names of each acyl chain.
        for lip_name in lipid_names:
            chain1_atom_names = natsort.natsorted(re.findall( r'C2[0-9]+', ','.join(list(np.unique(u.select_atoms('resname {}'.format(lip_name)).names)))))
            chain2_atom_names = natsort.natsorted(re.findall( r'C3[0-9]+', ','.join(list(np.unique(u.select_atoms('resname {}'.format(lip_name)).names)))))
            c_names_dict[lip_name] = {'c1': chain1_atom_names, 'c2': chain2_atom_names,
                                      'c1_string': "resname %s and "%lip_name + "(name " + " or name ".join(chain1_atom_names) + ")",
                                      'c2_string': "resname %s and "%lip_name + "(name " + " or name ".join(chain2_atom_names) + ")"}
            op_results[lip_name] = [[], []]
            op_results_overall[lip_name] = [[], []]
            op_results_empty = op_results.copy()

        with ThreadPoolExecutor() as executor:
            timeout = 5 # set the timeout in seconds
            futures = [executor.submit(cal_op, args) for args in u.trajectory[-20:]]
            for future in as_completed(futures):
                try:
                    result = future.result(timeout=timeout)
                    #print(result)
                except TimeoutError:
                    print("TimeoutError Occured, moving to the next task")
                    continue
                except StopIteration:
                    print("StopIteration Occured, moving to the next task")
                    
        for k, v in result.items():
            op_results_overall[k][0].append(np.nanmean(v[0], axis=0))
            op_results_overall[k][1].append(np.nanmean(v[1], axis=0))
#        with Pool() as pool:
#            for rst in pool.map(cal_op, u.trajectory[-200:]):
#                #time.sleep(.1)
#                if type(rst) is dict:
#                    #print("GOOD")
#                    for k, v in rst.items():
#                        op_results_overall[k][0].append(np.average(v[0], axis=0))
#                        op_results_overall[k][1].append(np.average(v[1], axis=0))
#                else:
#                    print("NONE")
#        with ProcessPool() as pool:
#            future = pool.map(cal_op, u.trajectory[-1000::10], timeout=10)
#            iterator = future.result()
#            while True:
#                try:
#                    result = next(iterator)
#                    if result != "":
#                        for k, v in rst.items():
#                            op_results_overall[k][0].append(np.average(v[0], axis=0))
#                            op_results_overall[k][1].append(np.average(v[1], axis=0))
#                except StopIteration:
#                    continue
        #time.sleep(TIMEOUT)
        #pool.terminate()
        
        
        
        print("Multiprocessing Finished")
        for lip_name in lipid_names:    
            op_results_overall[lip_name][0] = np.average(np.array(op_results_overall[lip_name][0]), axis=0)
            op_results_overall[lip_name][1] = np.average(np.array(op_results_overall[lip_name][1]), axis=0)

            for i in range(2):
                csv_data.append([folder, st, lip_name, 'SN'+str(i+1)] + list(op_results_overall[lip_name][i]))
    print(folder+' has been processed.')
    os.chdir('../../')

In [ ]:
futures[4].result()

In [ ]:

with open('op_DA_75_v1.csv', 'w', encoding='UTF8', newline='') as f:
    writer = csv.writer(f)

    # write the header
    writer.writerow(csv_header)

    # write multiple rows
    writer.writerows(csv_data)

---

### Contact fraction, $df$. For MARTINI simulations

---

## Terminal Methyl Group Distributions

In [ ]:
csv_header = ['composition', 'gamma', 'TMD_max']
csv_data = []

In [ ]:
density_type = 'Number'
n_bins = 200
lipid_name = "DOPC"
kg_to_u = 6.022e+26
m_to_a = 1e10
PA_names = ['DEPA','DGPA','DLPA','DMPA','DNPA','DOPA','DPPA','DYPA','PLPA','POPA','PSPA','SLPA','SOPA','YOPA']
PA_names = ['DLPA','DMPA','DPPA','DSPA']
DS_names = ['DSPC', 'DSPE', 'DSPG', 'DSPS', 'DSPA']

def aveNearTwo(array):
    a = array[:-1]
    b = array[1:]
    return ( a + b ) /2

def makeBilayerWhole(atom_pos, leaflet):
    global zedges, dens, center_of_blank
    bin_size = 10
    atom_pos_min = np.min(atom_pos[:,2])
    if np.ptp(atom_pos[:,2]) > boxz:
        while np.ptp(atom_pos[:,2]) > boxz:
            #print(np.ptp(atom_pos[:,2]))
            dens, zedges = np.histogram(atom_pos[:,2], bins= bin_size)
            center_of_blank = np.average(aveNearTwo(zedges)[dens == 0])   
            atom_pos = u.select_atoms("resname {} and (name {} or name {})".format(lipid_name, methyl_group_carbon_l1,methyl_group_carbon_l2 )).translate([0,0,-center_of_blank]).wrap()
            if np.ptp(atom_pos[:,2]) < boxz:
                break
            bin_size += 2
            #print(bin_size)

    
    elif leaflet == 0:
        atom_pos = u.select_atoms("resname {} and (name {} or name {})".format(lipid_name, methyl_group_carbon_l1,methyl_group_carbon_l2 )).translate([0,0,-atom_pos_min]).wrap()
    elif leaflet == 1:
        atom_pos = u.select_atoms("resid   1:100 and resname {} and (name {} or name {})".format(lipid_name, methyl_group_carbon_l1,methyl_group_carbon_l2)).translate([0,0,-atom_pos_min]).wrap()
    elif leaflet == 2:
        atom_pos = u.select_atoms("resid 101:200 and resname {} and (name {} or name {})".format(lipid_name,methyl_group_carbon_l1,methyl_group_carbon_l2 )).translate([0,0,-atom_pos_min]).wrap()
    else:
        raise ValueError(" Input Leaflet parameter invalid, must be 0,1,2.")


    atom_pos[:,2] -= np.average(atom_pos[:,2])
    
    return atom_pos

results = {}
    
    

for st in surf_tensions:
    results[st] = {}
    for sec_lipid_name in PA_names[:]:
        print("PROCESSING {}".format(sec_lipid_name), end='\r')
        os.chdir("/scratch/xu.jiam/liposomes/namd_sims/DOPC+PA/75-DOPC-{}-input/namd".format(sec_lipid_name))
        dcdfiles = get_files()
    
        
        results[st][sec_lipid_name] = {}
        dcdfile = [ i for i in dcdfiles if i.startswith('gamma'+str(st)) ]
        u = mda.Universe('step5_input.psf', dcdfile)
  
        carbon_ind_l1 = np.unique([ x for x in u.select_atoms('resname DOPC').names if re.search('^C2', x) ])
        carbon_ind_l2 = np.unique([ x for x in u.select_atoms('resname DOPC').names if re.search('^C3', x) ])
        methyl_group_carbon_l1 = natsort.natsorted(carbon_ind_l1, alg=natsort.ns.REAL)[-1]
        methyl_group_carbon_l2 = natsort.natsorted(carbon_ind_l2, alg=natsort.ns.REAL)[-1]
        ## ave_z, ave_y contains z and y methyl group positions of l1+l2, l1, l2, respectively in the last dimension.
        ave_z = np.zeros([len(u.trajectory[:]), n_bins])
        ave_y = np.zeros([len(u.trajectory[:]), n_bins, 3])

        for frame in u.trajectory[:]:
            
            #plt.plot(atom_pos[:,1], atom_pos[:,2], 'x')

            boxx = frame.dimensions[0]
            boxz = frame.dimensions[2]
            # Make bilayer whole.
            
            #atom_pos = makeBilayerWhole(atom_pos, 0)
            #atom_pos_l1 = makeBilayerWhole(atom_pos, 1)
            #atom_pos_l2 = makeBilayerWhole(atom_pos, 2)
            ### Only Z coordinates are used for density profile calculation.
            try:
                repair_bilayer(u, frame)
            except:
                print("Unable to process frame "+str(frame.frame))
                continue

            atom_pos = u.select_atoms("resname {} and (name {} or name {})".format(lipid_name, methyl_group_carbon_l1, methyl_group_carbon_l2)).positions[:,2]
            atom_pos_l1 = u.select_atoms("resname {} and (name {} or name {}) and resid 1-100".format(lipid_name, methyl_group_carbon_l1, methyl_group_carbon_l2)).positions[:,2]
            atom_pos_l2 = u.select_atoms("resname {} and (name {} or name {}) and resid 101-200".format(lipid_name, methyl_group_carbon_l1, methyl_group_carbon_l2)).positions[:,2]

            unit_volume = boxx ** 2 * (np.max(atom_pos) - np.min(atom_pos))/(n_bins-1) # in A^3

            #plt.plot(atom_pos[:,1], atom_pos[:,2], 'x')
            if density_type == 'Number':
                binz = np.linspace(0, boxz, n_bins+1)
                hst = np.histogram(atom_pos, bins= binz)
                hst_l1 = np.histogram(atom_pos_l1, bins= binz)
                hst_l2 = np.histogram(atom_pos_l2, bins= binz)
                if sum(hst[0]) != 0:
                    ave_z[frame.frame, :   ]=(hst[1][1:] + hst[1][:-1])/2
                    ave_y[frame.frame, :, 0]=hst[0]  / (binz[-1] - binz[0] ) 
                    ave_y[frame.frame, :, 1]=hst_l1[0] / (binz[-1] - binz[0] ) 
                    ave_y[frame.frame, :, 2]=hst_l2[0]  / (binz[-1] - binz[0] ) 
            
            elif density_type == 'Mass':
                atom_mass = u.select_atoms("resname {}".format(lipid_name)).masses
                binz = np.linspace(-0.5 * boxz, 0.5*boxz, n_bins+1)
                biny = [0, 100]
                hist_sum_mass = binned_statistic_2d(atom_pos, atom_mass, atom_mass, 'sum', bins=[binz, biny])
                if sum(hist_sum_mass[0]) != 0:
                    ave_z[frame.frame,:]=(hist_sum_mass[1][1:] + hist_sum_mass[1][:-1])/2
                    ave_y[frame.frame,:]=np.ravel(hist_sum_mass[0]) * m_to_a / (binz[-1] - binz[0] ) / kg_to_u
        results[st][sec_lipid_name]['z'] = ((np.average(ave_z, axis=0)-np.average(ave_z))[20:-20]).copy()
        results[st][sec_lipid_name]['y'] = (np.average(ave_y[:,:], axis=0)[20:-20]).copy()
    print(" --- SURFACE TENSION " + str(st) + " DONE --- ")

In [ ]:
for st in surf_tensions:
    for PA in PA_names[:]:
        csv_data.append(['85-DOPC-'+PA, st, np.max(results[st][PA]['y'][:,0])])
        

In [ ]:
os.chdir('/scratch/xu.jiam/liposomes/namd_sims/DOPC+PA')
with open('TMD_max.csv', 'w', encoding='UTF8', newline='') as f:
    writer = csv.writer(f)

    # write the header
    writer.writerow(csv_header)

    # write multiple rows
    writer.writerows(csv_data)

In [ ]:
os.chdir('/scratch/xu.jiam/liposomes/namd_sims/DOPC+PA/')
fig, axs = plt.subplots(nrows=4, ncols=1, figsize=(6, 15))
i=0
j = 0
colors = ['gray', 'blue', 'red', 'green', 'brown']
for st in surf_tensions:
    for sec_lipid_name in PA_names[:]:
        axs[i].plot(results[st][sec_lipid_name]['z'], results[st][sec_lipid_name]['y'][:,1] + results[st][sec_lipid_name]['y'][:,2], alpha=1, label = sec_lipid_name, color = colors[j])
        axs[i].plot(results[st][sec_lipid_name]['z'], results[st][sec_lipid_name]['y'][:,1], alpha=0.7, color = colors[j])
        axs[i].plot(results[st][sec_lipid_name]['z'], results[st][sec_lipid_name]['y'][:,2], alpha=0.7, color = colors[j])
        j += 1
    j = 0
    i += 1
plt.legend()
plt.savefig('TMD_75_25PA.jpg', dpi=300)
#plt.show()
plt.close()

## Calculate pairInteractionEnergies for NAMD trajectories

In [8]:

os.chdir('/work/hung_group/xu.jiam/liposomes/namd_sims/DOPC+PA/75-DOPC-DSPA-input/namd')


### Update pdb files for head, bone, tail, and solvent groups

In [2]:
def update_pdb_Bfactor(filename):
    """
    This function updates B factor column of its input pdb file with following rules:
    Add headgroup, phosphate group, glycerol group, and tail group for lipid with PS, PA, PC, PE, PG in its residue name.
    Add individual groups for other residues.
    
    Input filename: file name of raw pdb file
    Return: a dictionary for B factor index and its corresponding group
    
    Note: New pdb file will be generated directly under current folder, with _B_factor addition before ".pdb".
    
    """
    
    u = mda.Universe(filename)
    selection_strings = {}

    ### ----- Generate selection strings  ----- ###

    for rs in pd.unique(u.atoms.resnames):
        # regex checks if it's a lipid
        if re.search('(PC|PA|PE|PS|PG)', rs):
            lipid_atom_names = pd.unique(u.select_atoms('resname '+rs).names)
            P_index = np.where(lipid_atom_names == 'P')[0][0]
            C1_index = np.where(lipid_atom_names == 'C1')[0][0]
            C32_index = np.where(lipid_atom_names == 'C32')[0][0]
            if P_index != 0:
                selection_strings[rs+'_head_string'] = 'resname ' + rs + ' and (name ' + ' or name '.join(lipid_atom_names[:P_index]) + ')'
            selection_strings[rs+'_phos_string'] = 'resname ' + rs + ' and (name ' + ' or name '.join(lipid_atom_names[P_index:C1_index]) + ')'
            selection_strings[rs+'_glyc_string'] = 'resname ' + rs + ' and (name ' + ' or name '.join(lipid_atom_names[C1_index:C32_index]) + ')'
            selection_strings[rs+'_tail_string'] = 'resname ' + rs + ' and (name ' + ' or name '.join(lipid_atom_names[C32_index:]) + ')'
        else:
            selection_strings[rs] = 'resname ' + rs

    ### ----- Update B group  ----- ###

    ft = 1
    bf_group = {}
    
    for k, v in selection_strings.items():
        u.select_atoms(v).tempfactors = ft
        
        if '_' in k:
            bf_group["_".join(k.split("_")[:-1])] = ft
        else:
            bf_group[k] = ft
        ft += 1

    ### ----- Save updated pdb file ----- ###
    insert_string = "_B_factor."
    output_string = re.sub(r'\.(?=[^\.]*$)', insert_string, filename)
    print("Processed pdb file has been saved to "+output_string)
    u.atoms.write(output_string)
    
    return bf_group



In [4]:
rerun_group_string = """&\npairInteraction         on  ;&\npairInteractionFile     step5_input_B_factor.pdb  ;&\npairInteractionSelf     on  ;&\npairInteractionCol      B   ;&\npairInteractionGroup1   1  ;&\npairInteractionGroup2   1  ;&\n"""
rerun_tcl_string = """
coorfile open dcd ./gamma%i_total.dcd
set ts 1000
# Read all frames until nonzero is returned.
while { ![coorfile read] } {
  # Set firstTimestep so our energy output has the correct TS.
  firstTimestep $ts
  # Compute energies and forces, but don't try to move the atoms.
  run 0
  incr ts 10000
}
coorfile close
"""

In [ ]:
if __name__ == "__main__":
    surf_tensions = [-7, 0, 7, 15]
    for st in surf_tensions[:1]:
        os.system(f"mdconvert gamma{st}.*.dcd -c 100000 -o gamma{st}_total.dcd")
        input_file = get_files(pre=f'gamma{st}', ext='inp')[0]
        os.system(f"sed -i 's/numsteps     /s/.*/numsteps    0; /g' {input_file}")
        os.system(f"sed -i 's/run       /s/.*/run      0;/g' {input_file}")
        os.system(f"sed -i '/run      0;/,$d/' {input_file}")
        os.system(f"sed -i '/seed /s/.*/seed 123;/g' {input_file}")
        p = subprocess.Popen(["grep", "-n", "source", "gamma-7_1.inp"], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        stdout, stderr = p.communicate()
        line_2_insert = int(stdout.decode().split(':')[0])+1
        os.system(f"sed -i '{line_2_insert}a\\{rerun_group_string}/' {input_file}")
        # Add rerun commands.
        with open(input_file, 'a') as f:
            print(rerun_tcl_string%(st), file=f)
        # update pdb file's B factor
        b2g = update_pdb_Bfactor('step5_input.pdb')
        for group1_name, group1_ndx in b2g.items():
            for group2_name, group2_ndx in b2g.items():
                # Change group index
                os.system("sed -i '/pairInteractionGroup1 /s/.*/pairInteractionGroup1  {}/g' {}".format(group1_ndx, input_file))
                os.system("sed -i '/pairInteractionGroup2 /s/.*/pairInteractionGroup2  {}/g' {}".format(group2_ndx, input_file))
                os.system("charmrun +p27 namd2 {0} > gamma{1}_{2}_{3}.log".format(input_file, st, group1_name, group2_name))

In [26]:
import subprocess
subprocess.Popen(["grep -n","'source'", "gamma-7_1.inp"])

FileNotFoundError: [Errno 2] No such file or directory: 'grep -n'

In [13]:
os.chdir('/work/hung_group/xu.jiam/liposomes/namd_sims/DOPC+PA/75-DOPC-DOPA-input/namd')

In [30]:
p = subprocess.Popen(["grep", "-n", "source", "gamma-7_1.inp"], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
stdout, stderr = p.communicate()

In [35]:
int(stdout.decode().split(':')[0])+1

79

In [ ]:

os.system("module load openmpi/4.0.5-skylake-gcc10.1")
os.system("module load gcc/10.1.0")
os.system("module load namd/2.14-mpi")

In [ ]:
u.select_atoms(selection_strings['DOPC_head_string'])

In [ ]:
P_index

In [ ]:
!module {list}